# Demokrasi gibi düşün. En güçlü konsensus KAZANIR

# Ensemble Learning(Topluluk Öğrenmesi)
* Ensemble Learning birden fazla makine öğrenmesi modelini birleştirerek daha iyi tahmin performansı elde etmeyi amaçlayan bir yaklaşımdır.Temek fikir tek bir model yerine birden fazla modeli bir araya getirerek daha kararlı ve doğru sonuçlar elde etmektir

## Avantajları
* Daha yüksek tahmin doğruluğu
* Overfitting riskini azaltma
* Farklı model türlerinin güçlü yönlerini birleştirme
* Model stabilitesini arttırma


## Ensemble Yöntemleri

### 1-)Bagging (Bootstrap AGGregatING)
** Bagging yöntemi aynı algoritmanın bootstrap örneklemi adı verilen farklı veri alt kümelerinde eğitilmesine dayanır.Orijinal veri setinden rastgele örnekler seçerek(yerine koyarak) birçok veri alt kümesi oluşturulur.Her model bir alt küme üzerinde eğitilir ve sonuçlar birleştirilir(Sonuçlar oylanarak(kümeleme) ya da ortalama alınarak(regresyon) birleştirilir)
* Random Forest baggingin en popüler örneğidir.Her düğümde rastgele seçilmiş özelliklerin alt kümesini kullanarak birden çok karar ağacı eğitilir
* Amacı varyansı azaltmaktır. Özellikle yüksek varyansa sahip modeller için faydalıdır

** Bagging matematiği bias-variance trade-off ile çalışır
$$Var[\hat{f}_{bag}] = \frac{1}{B^2} \sum Cov(\hat{f}_i, \hat{f}_j)$$

* B=toplam model sayısı

### 2-)Boosting
** Boosting zayıf öğrencileri ardışık şekilde eğitir.Her yeni model bir önceki modelin yanlış sınıflandırdığı örneklere daha fazla odaklanarak birbirlerinin hatalarını düzeltmeye çalışır
* Amacı biası azaltmaktır

** Popüler boosting algoritmaları
* AdaBoost--> Yanlış sınıflandırılan örneklerin ağırlıklarını arttırır.
* Gradient Boosting--> Hatayı minimize etmek için gradyan(eğim) inişi kullanır
* XGBoost--> Hız ve performans için optimize edilmiştir
* LightGBM--> Büyük veri setleri için daha hızlıdır
* CatBoost--> Kategorik değişkenlerle daha iyi çalışır

** Boosting Matematiği
$$F_m(x) = F_{m-1}(x) + \gamma_m h_m(x)$$

* Fm-1(x):önceki model
* m(x):yeni öğrenilen zayıf model
* γm:Öğrenme oranı


### 3-)Stacking
* Stacking farklı model türlerinin(örneğin bir logistic regresyon, bir decision tree,bir SVM) tahminlerini bir araya getirerek bir "meta öğrenici" eğitir.Bu öğrenici diğer modellerin çıktılarını girdi olarak alır ve son tahmini üretir.

** Süreç
* Birinci seviye modelleri(base learners) eğitin
* Birinci seviye modellerin tahminlerini kullanarak ikinci seviye modeli(meta-learner) eğitin
* Test veirleri için önce birince seviye modellerin tahminlerini alın,sonra bu tahminleri meta-learner'a girdi olarak verin


### 4-)Voting
Voting, birkaç modelin tahminlerini birleştirmenin en basit yoludur
* Hard Voting:Sınıflandırma için çoğunluk oylaması yapar(en çok oy alan sınıf seçilir)
* Soft Voting:Sınıflandırma için sınıf olasılıklarının ortalamasını alır
* Regresyon için Voting:Tahminlerin ortalaması alınır

## Ensemble Yöntemlerinde Hiperparametre Optimizasyonu
* n_estimators:Toplulukta kullanılacak model sayısı
* learning rate:Boosting algoritmalarında her modelin katkısını ölçeklendiren değer
* max_depth:Ağaç modellerinde max derinlik
* subsample:Her iterasyonda kullanılacak veri oranı
* colsample_bytree:Her ağaç için kullanılacak özellik oranı

## Ensemble Learning için Pratik İpuçları
* Tek bir güçlü model ile başlayıp,benchmark olarak kullanın
* Grid Search veya Random Search ile hiperparametre optimizasyonu yapın
* Farklı model türlerini birleştir(doğrusal+ağaç tabanlı)
* Aşırı karmaşık ensemble'lar her zaman daha iyi sonuç vermeyebilir

# Ensemble Learning Kod Örnekleri

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,VotingClassifier,AdaBoostClassifier,StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
import warnings
warnings.filterwarnings("ignore")

# -----Örnek 1-)BAGGING(RANDOM FOREST)------

In [2]:
#Breast Cancer veri setini yükle
cancer=load_breast_cancer()
X=pd.DataFrame(cancer.data,columns=cancer.feature_names)
y=cancer.target
print("Veri seti boyutu",X.shape)
print("Sınıf dağılımı:\n",pd.Series(y).value_counts())

Veri seti boyutu (569, 30)
Sınıf dağılımı:
 1    357
0    212
Name: count, dtype: int64


In [3]:
#Veriyi eğitim ve test olarak ayıralım
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=31)
#Baggin örneği random forest modelini oluştur ve fit et
rf_model=RandomForestClassifier(n_estimators=100,random_state=31)
rf_model.fit(X_train,y_train)
#Test verisi üzerinden tahmin yap
y_pred=rf_model.predict(X_test)
print("Random Forest Performansı:")
print("Doğruluk",accuracy_score(y_test,y_pred))
print("-----"*10)
print(classification_report(y_test,y_pred))


Random Forest Performansı:
Doğruluk 0.9440559440559441
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.94      0.91      0.93        56
           1       0.94      0.97      0.95        87

    accuracy                           0.94       143
   macro avg       0.94      0.94      0.94       143
weighted avg       0.94      0.94      0.94       143



In [4]:
#Özellik önemlerini görelim
feature_importance=pd.DataFrame({"Feature":X.columns,
                                 "Importance":rf_model.feature_importances_}).sort_values("Importance",ascending=False)
print("En önemli 6 özellik:")
feature_importance.head(6)

En önemli 6 özellik:


,Feature,Importance
20,worst radius,0.136938
23,worst area,0.125543
27,worst concave points,0.109565
22,worst perimeter,0.104903
3,mean area,0.093277
0,mean radius,0.083438


# -----Örnek 2-)BOOSTING(Gradient Boosting)-----

In [5]:
#Gradient Boosting(Gradyan Arttırma) modelini oluştur ve eğit
gb_model=GradientBoostingClassifier(n_estimators=100,learning_rate=0.1,random_state=31)
gb_model.fit(X_train,y_train)
#test verisi üzerinden tahmin
y_pred=gb_model.predict(X_test)
print("Gradient Boosting Performansı")
print("Doğruluk",accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

Gradient Boosting Performansı
Doğruluk 0.9790209790209791
              precision    recall  f1-score   support

           0       0.96      0.98      0.97        56
           1       0.99      0.98      0.98        87

    accuracy                           0.98       143
   macro avg       0.98      0.98      0.98       143
weighted avg       0.98      0.98      0.98       143



## ----- Örnek 3-) Voting Classifier -----

In [6]:
#farklı modeller oluştur
log_clf=LogisticRegression(random_state=31)
rf_clf=RandomForestClassifier(n_estimators=100,random_state=31)
svm_clf=SVC(probability=True,random_state=31)

#voting classifier oluştur
voting_clf=VotingClassifier(estimators=[
    ("lr",log_clf),
    ("rf",rf_clf),
    ("svc",svm_clf)
],voting="soft")
#modeli eğit
voting_clf.fit(X_train,y_train)
#test verisi üzerinden tahmin yap
y_pred=voting_clf.predict(X_test)
print("Voting Classifier Performansı")
print("Doğruluk",accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

Voting Classifier Performansı
Doğruluk 0.9440559440559441
              precision    recall  f1-score   support

           0       1.00      0.86      0.92        56
           1       0.92      1.00      0.96        87

    accuracy                           0.94       143
   macro avg       0.96      0.93      0.94       143
weighted avg       0.95      0.94      0.94       143



## ----- Örnek 4-) STACKING CLASSIFIER -----

In [7]:
#base modelleri tanımla
base_models=[
    ("rf",RandomForestClassifier(n_estimators=50,random_state=31)),
     ("gb",GradientBoostingClassifier(n_estimators=50,random_state=31)),
     ("dt",DecisionTreeClassifier(random_state=31))]

#Meta Learner Oluştur
meta_learner=LogisticRegression(random_state=31)
#Stacking Classifier oluştur
stacking_clf=StackingClassifier(estimators=base_models,
                                final_estimator=meta_learner,
                                cv=5)
#modeli eğit
stacking_clf.fit(X_train,y_train)
#tahmin yap
y_pred=stacking_clf.predict(X_test)
print("Stacking Classifier Performansı")
print("Doğruluk:",accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

Stacking Classifier Performansı
Doğruluk: 0.9790209790209791
              precision    recall  f1-score   support

           0       0.96      0.98      0.97        56
           1       0.99      0.98      0.98        87

    accuracy                           0.98       143
   macro avg       0.98      0.98      0.98       143
weighted avg       0.98      0.98      0.98       143



## ----- Örnek 5-) ADABOOST-----

In [8]:
#AdaBoost modelini oluştur ve eğit
ada_model=AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),#base estiamtor karar ağacı
                             n_estimators=200,
                             algorithm="SAMME",
                             learning_rate=0.8,
                             random_state=31)
ada_model.fit(X_train,y_train)
#tahmin
ada_model.predict(X_test)
print("AdaBoost Performansı")
print("Doğruluk",accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

AdaBoost Performansı
Doğruluk 0.9790209790209791
              precision    recall  f1-score   support

           0       0.96      0.98      0.97        56
           1       0.99      0.98      0.98        87

    accuracy                           0.98       143
   macro avg       0.98      0.98      0.98       143
weighted avg       0.98      0.98      0.98       143



# MODELLER ARASI KARŞILAŞTIRMA

In [9]:


models = [
    ("Random Forest", rf_model),
    ("Gradient Boosting", gb_model),
    ("Voting Classifier", voting_clf),
    ("Stacking Classifier", stacking_clf),
    ("AdaBoost", ada_model)
]
print("Modeller arası karşılaştırma(10 Fold cross validation)")
for name,model in models:
    scores=cross_val_score(model,X,y,cv=10,scoring="accuracy")
    print(f"{name}: Ortalama Doğruluk = {scores.mean():.4f}, Standart Sapma = {scores.std():.4f}")

Modeller arası karşılaştırma(10 Fold cross validation)
Random Forest: Ortalama Doğruluk = 0.9596, Standart Sapma = 0.0294
Gradient Boosting: Ortalama Doğruluk = 0.9632, Standart Sapma = 0.0254
Voting Classifier: Ortalama Doğruluk = 0.9543, Standart Sapma = 0.0250
Stacking Classifier: Ortalama Doğruluk = 0.9596, Standart Sapma = 0.0249
AdaBoost: Ortalama Doğruluk = 0.9754, Standart Sapma = 0.0141


<h1 style="color:red">California Housing Veri Seti ile Regresyon Örneği</h1>

In [10]:
#gerekli kütüphaneleri yükle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,VotingRegressor,StackingRegressor,AdaBoostRegressor
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

In [21]:
#veri setini yükle
df=fetch_california_housing()
X=pd.DataFrame(df.data,columns=df.feature_names)
y=df.target
print("Veri seti boyutu :",X.shape)
print("Hedef değişken istatistikleri:")
print("Min y:",np.min(y))
print("Max  y:",np.max(y))
print("Std y:",np.std(y))
print("Mean y:",np.mean(y))

Veri seti boyutu : (20640, 8)
Hedef değişken istatistikleri:
Min y: 0.14999
Max  y: 5.00001
Std y: 1.1539282040412253
Mean y: 2.068558169089147


In [24]:
#Veriyi eğitim ve test olarak böl
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=31)
#Özellileri ölceklendirme(scaling)
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

### ----- Örnek 1-)Random Forest Regressor ----

In [39]:
rf_reg=RandomForestRegressor(n_estimators=100,random_state=31)
rf_reg.fit(X_train_scaled,y_train)
y_pred_rf=rf_reg.predict(X_test_scaled)
print("MSE :",mean_squared_error(y_test,y_pred_rf))
print("RMSE :",np.sqrt(mean_squared_error(y_test,y_pred_rf)))
print("MAE :",mean_absolute_error(y_test,y_pred_rf))
print("R2_Score :",r2_score(y_test,y_pred_rf))

#Özellik Önemlerini Görselleştir
feature_importance=pd.DataFrame({"Feature":X.columns,
                                 "Importance":rf_reg.feature_importances_}).sort_values("Importance",ascending=False)
print("En önemli 5 özellik")
feature_importance.head(5)

MSE : 0.2591044361296861
RMSE : 0.5090230212177894
MAE : 0.33042886237887614
R2_Score : 0.8025485781030028
En önemli 5 özellik


,Feature,Importance
0,MedInc,0.522559
5,AveOccup,0.136851
7,Longitude,0.088541
6,Latitude,0.087652
1,HouseAge,0.054186


### ----- Örnek 2-)Gradient Boosting Regressor -----

In [46]:
gb_reg=GradientBoostingRegressor(n_estimators=100,learning_rate=0.1,random_state=31)
gb_reg.fit(X_train_scaled,y_train)
y_pred_gb=gb_reg.predict(X_test_scaled)
print("MSE :",mean_squared_error(y_test,y_pred_gb))
print("RMSE :",np.sqrt(mean_squared_error(y_test,y_pred_gb)))
print("MAE :",mean_absolute_error(y_test,y_pred_gb))
print("R2_Score :",r2_score(y_test,y_pred_gb))

MSE : 0.2881242745550127
RMSE : 0.5367720880923418
MAE : 0.36937518155171867
R2_Score : 0.780433910959929


### ----- Örnek 3-)Voting Regressor -----

In [ ]:
#Farklı regresyon modelleri oluştur
lr=LinearRegression()
rf=RandomForestRegressor(n_estimators=100,random_state=31)
gbr=GradientBoostingRegressor(n_estimators=100,random_state=31)
#Voting Regressor oluştur
voting_reg=VotingRegressor([
    ("lr",lr),
    ("rf",rf),
    ("gbr",gbr)
])
voting_reg.fit(X_train_scaled,y_train)
y_pred_voting=voting_reg.predict(X_test_scaled)
print("MSE :",mean_squared_error(y_test,y_pred_voting))
print("RMSE :",np.sqrt(mean_squared_error(y_test,y_pred_voting)))
print("MAE :",mean_absolute_error(y_test,y_pred_voting))
print("R2_Score :",r2_score(y_test,y_pred_voting))


MSE : 0.2998539292592458
RMSE : 0.5475891975370276
MAE : 0.37880365811775996
R2_Score : 0.7714952874677692


### ----- Örnek 4-) Stacking Regressor ----


In [49]:
#Base estimatorları belirle
estimators=[("rf",RandomForestRegressor(n_estimators=100,random_state=31)),
            ("gbr",GradientBoostingRegressor(n_estimators=100,random_state=31)),
            ("svr",SVR(kernel="rbf"))]
# Stacking Regressor oluştur
stacking_reg=StackingRegressor(estimators=estimators,
                               final_estimator=Ridge(random_state=31))

stacking_reg.fit(X_train_scaled,y_train)
y_pred_stacking=stacking_reg.predict(X_test_scaled)
print("MSE :",mean_squared_error(y_test,y_pred_stacking))
print("RMSE :",np.sqrt(mean_squared_error(y_test,y_pred_stacking)))
print("MAE :",mean_absolute_error(y_test,y_pred_stacking))
print("R2_Score :",r2_score(y_test,y_pred_stacking))

MSE : 0.25509818815503393
RMSE : 0.5050724583216095
MAE : 0.33038646010080236
R2_Score : 0.8056015530766584


### ----- Örnek 5-) AdaBoost Regressor -----

In [51]:
ada_reg=AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
                          n_estimators=100,
                          learning_rate=0.1,
                          random_state=31)

ada_reg.fit(X_train_scaled,y_train)
y_pred_ada=ada_reg.predict(X_test_scaled)
print("MSE :",mean_squared_error(y_test,y_pred_ada))
print("RMSE :",np.sqrt(mean_squared_error(y_test,y_pred_ada)))
print("MAE :",mean_absolute_error(y_test,y_pred_ada))
print("R2_Score :",r2_score(y_test,y_pred_ada))

MSE : 0.4771205088858854
RMSE : 0.6907391033421268
MAE : 0.5482875564570011
R2_Score : 0.6364086840698313
